# <span style="color:blue">EVCO Workshop 5: Rule-Based GP in DEAP</span>

Simon O'Keefe: simon.okeefe@york.ac.uk

Danny Roberts: danny.roberts@york.ac.uk

Tianda Sun: tianda.sun@york.ac.uk

You have already experienced how to use GP, so there is no walkthrough ahead of this workshop. Here, you will learning to program GP to produle a simple rule-based AI and deal with bloat.

# <span style="color:#0073e6">Topics</span>

- Introduction to solving rule-based problems with GP, including an intricate system of functions and resources 
- Implementing a loosely-typed GP to solve the classic Santa Fe Ant Trail problem
- Dealing with bloat

# <span style="color:#0073e6">Learning Objectives</span>

- Learn how to program a GP to produce rule-based AI
- Understand and deal with bloat

# <span style="color:Blue">Exercise: Santa Fe Ant Trail Problem</span>

# <span style="color:#0073e6">Context</span>

The artificial ant problem (or Santa Fe Ant Problem) is a well-studied problem with a simple fitness function. The problem is to devise an agent that can successfully navigate an artificial ant along a trail on a square grid to collect as much food as possible within a maximum number of moves (here 600, but you can increase it). You can see a visualization of the problem here: 
https://www.youtube.com/watch?v=BKF7pGw8qbY

# <span style="color:#0073e6">Map Setup</span>

Make sure that you have the file santafe_trail.txt in the same folder as the notebook. This text file includes the map for the ants to solve. Open the map in a text editor. You can see that empty cells are represented with periods and food is represented with a hash. There is only one map to work with here, but you can create your own varitions of this in your text editor (e.g. even use code to flip the map).

# <span style="color:#0073e6">The Ant Class</span>

In the code, an ant will follow the instructions provided to it from an individual (an evolved decision tree) evolved by GP. So there is a single ant agent that is used to evaluate your individuals, but a population of individuals that use the ant for evaluation.  A basic way to evaluate fitness is simply looking at how much a particular ant has eaten (ant.eaten), which is implemented here.

An ant can navigate based on three operations:
- Move_Forward	- Move the ant forward one square
- Turn_Right		- Turn the ant to the right 90 degrees
- Turn_Left		- Turn the ant to the left 90 degrees.

These three commands therefore part of the terminal set in a GP. Each of these operations is assumed to take one time unit to execute.

The sensing function if_food_ahead looks into the square the ant is currently facing and then executes one of its two arguments depending upon whether that square contains food or is empty. Two other functions, Prog2 and Prog3, are provided. These take two and three arguments respectively which are executed in sequence. These three functions are therefore also part of the primitive set.

# <span style="color:Blue">Exercise 1: Complete the GP algorithm</span>

1. Go through the code below and try to understand what it the ant is doing and how the problem works.
2. Complete the genetic programming algorithm to find a solution (as in previous examples). This mainly means completing the primitive set and registering fitness and operators.
3. The best solution in terms of number of food items eaten is 89 for this problem. Can you improve the performance of your algorithm? Be sure to also output your final best solution as tree for examination.

# <span style="color:#0073e6">Ant Code</span>

In [3]:
import copy
import random
import numpy
# import pygraphviz as pgv
from functools import partial

from deap import base
from deap import creator
from deap import tools
from deap import gp

In [4]:
class AntSimulator(object):
    direction = ["north","east","south","west"]
    dir_row = [1, 0, -1, 0]
    dir_col = [0, 1, 0, -1]
    
    def __init__(self, max_moves):
        self.max_moves = max_moves
        self.moves = 0
        self.eaten = 0
        self.routine = None
        
    def _reset(self):
        self.row = self.row_start 
        self.col = self.col_start 
        self.dir = 1
        self.moves = 0  
        self.eaten = 0
        self.matrix_exc = copy.deepcopy(self.matrix)

    @property
    def position(self):
        return (self.row, self.col, self.direction[self.dir])
            
    def turn_left(self): 
        if self.moves < self.max_moves:
            self.moves += 1
            self.dir = (self.dir - 1) % 4

    def turn_right(self):
        if self.moves < self.max_moves:
            self.moves += 1    
            self.dir = (self.dir + 1) % 4
        
    def move_forward(self):
        if self.moves < self.max_moves:
            self.moves += 1
            self.row = (self.row + self.dir_row[self.dir]) % self.matrix_row
            self.col = (self.col + self.dir_col[self.dir]) % self.matrix_col
            if self.matrix_exc[self.row][self.col] == "food":
                self.eaten += 1
            self.matrix_exc[self.row][self.col] = "passed"

    def sense_food(self):
        ahead_row = (self.row + self.dir_row[self.dir]) % self.matrix_row
        ahead_col = (self.col + self.dir_col[self.dir]) % self.matrix_col        
        return self.matrix_exc[ahead_row][ahead_col] == "food"
   
    def if_food_ahead(self, out1, out2):
        return partial(if_then_else, self.sense_food, out1, out2)
   
    def run(self,routine):
        self._reset()
        while self.moves < self.max_moves:
            routine()
    
    def parse_matrix(self, matrix):
        self.matrix = list()
        for i, line in enumerate(matrix):
            self.matrix.append(list())
            for j, col in enumerate(line):
                if col == "#":
                    self.matrix[-1].append("food")
                elif col == ".":
                    self.matrix[-1].append("empty")
                elif col == "S":
                    self.matrix[-1].append("empty")
                    self.row_start = self.row = i
                    self.col_start = self.col = j
                    self.dir = 1
        self.matrix_row = len(self.matrix)
        self.matrix_col = len(self.matrix[0])
        self.matrix_exc = copy.deepcopy(self.matrix)

The 'parse_matrix' function processes the map and stores it in the 2D list 'matrix_exc', and the other functions simply move or turn the ant around.

*Side note: in case you're not familiar @property is a standard Python decorator that allows you do to setting and getting variables. It's beyond the scope of the module to discuss this, and you don't need to understand it for this workshop. If you're interested you can read more here:
https://www.programiz.com/python-programming/property*

In [5]:
ant = AntSimulator(600) # This is the number of moves

with  open("santafe_trail.txt") as trail_file:
    ant.parse_matrix(trail_file)

## <span style="color:#0073e6">You need to decide on your GP below</span>

We now need to define our primitive set. I see a lot of mistakes when defining the "Main" function here. That is because the ant uses information from the environment, and students add these as arguments. However, if you look at the code, the ant object directly accesses the environment, and it is not passed to the evolved function as an argment. As such, we have zero arguments here. However, you could modify the code to pass the arguments, if you liked, and then would need to modify it here.

Don't forget that here you might need to define and register progs (allowing multiple functions to connect together) and your own functions (e.g. if_then_else). No cheating and registering pre-programmed behaviour, though!

Your terminals are the ant behaviour. So for example:
> pset.addTerminal(ant.move_forward)

I have left a couple of examples in here to get you started with the primitive set.

In [ ]:
def progn(*args):
    for arg in args:
        arg()

def prog2(out1, out2): 
    return partial(progn,out1,out2) #partial allows this to be a more specific (two argument) version of progn

pset.addPrimitive(prog2, 2)
pset.addTerminal(ant.move_forward)

For the **evaluation function** for this task, you can see that the ant class has a routine to run through your compiled solution a number of timesteps.

# <span style="color:Blue">Exercise 2: Dealing with Bloat</span>

You should see from plotting the tree of the best individual, that there is bloat. Let's examine how to deal with this.

1. Try a crude modification to the evaluation function to add a ‘parsimony penalty’. This is a penality that increases as a function of the height of the tree ( individual.height ).
2. Now remove the bloat penalty. If parsimony penalties like this are not done carefully for some problems, it could give undesirable results. Instead implement a double tournament. Go to the DEAP library reference (https://deap.readthedocs.io/en/master/api/index.html) and look for the selection operator selDoubleTournament(). This will vastly help you to contain bloat in a better manner. Implement this, and play with the parameters a little. Then have a look at the final tree structure compared to before. This is also explained in the lecture.
3. Double tournaments typically work well when depth limits are also used with operators.  

For example:  
```toolbox.decorate("mate", gp.staticDepthLimit(20))
toolbox.decorate("mutate", gp.staticDepthLimit(20))```  

Which will limit the height of all individuals produced to 20 at most. 20 might be a good compromise between search space exploration and bloat control for a problem. However, you may adapt this threshold to your problem. You can decorate the mutation operator too. Note that some level of bloat is needed, because evolution often makes use of or builds upon redundancy.

## <span style="color:#0073e6">A note on the maximum tree depth in DEAP</span>

Since DEAP uses the Python parser to compile the code represented by the trees, it inherits from its limitations. The most commonly encountered restriction is the parsing stack limit. The Python interpreter parser stack limit is usually fixed between 92 and 99. This means that an expression can at most be composed of 91 succeeding primitives. In other words, a tree can have a maximum depth of 91 in DEAP.

# <span style="color:Blue">Bonus Practical: Strongly-typed GP</span>

If you would like an additional task, you can implement Strongly-Typed GP. Go through the Spambase problem at:
http://deap.readthedocs.io/en/master/examples/gp_spambase.html

## <span style="color:#0073e6">Bonus Ant Task</span>

If you like, you can display the map as the game unfolds. You could either use the Curses library to do this on the terminal, or Turtle library to display it in a window. However, this is optional and it will not be possible to offer support for it in the workshop time (more on this later, though).